In [1]:
import tensorflow
#import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten

#Creiamo una CNN, la quale avrà tre fasi: convoluzione, pooling e appiattimento
classifier = Sequential()

# CONVOLUTION
#Qui stiamo stabilendo come deve avvenire la convoluzione, ovvero vogliamo suddividere i pixel delle immagini secondo dei filtri.
#Si fa con Conv2D, in cui specifichiamo il numero di filtri = 32 e quanto deve essere grande la matrice dei filtri = 4 x 4. N.B. Li ho scelti a caso.
#input_shape indica con che forma ci si aspetta l'input, nel nostro caso sono foto 200 x 200. 3 è un numero che ogni pixel è combinazione di 3 valori RGB.
#Infine si sceglie la funzione di attivazione, in questo caso la RELU.
classifier.add(Conv2D(32, (4, 4), input_shape = (200, 200, 3), activation = 'relu'))

# POOLING
#Nella fase di pooling scegliamo come devono essere creati i pool a partire dalle matrici ottenute dalla convoluzione. 
#MaxPooling significa dire che si sceglie per ogni regione della matrice di convoluzione il valore massimo, e questo viene inserito nella nuova matrice di pooling.
#pool_size 2 x 2 indica ogni quanto deve trovare e inserire un valore massimo dalla matrice di convoluzione.
classifier.add(MaxPooling2D(pool_size = (6,6)))

#FLATTENING
#Bisogna trasformare tutte le matrici ottenute dalla fase di pooling in array monodimensionali, poiché questa è una forma di input accettabile da una rete neurale.
#Una funzione fa tutto da sola.
classifier.add(Flatten())


classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 197, 197, 32)      1568      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0         
Total params: 1,568
Trainable params: 1,568
Non-trainable params: 0
_________________________________________________________________


In [2]:
#FULL CONNECTION
#Ora bisogna creare l'effettiva rete neurale. Bisogna crearne una classica multistrato alimentata in avanti.
#La dimensione dell'input sarà la dimensione dell'output creato dalla fase di appiattimento, quindi 307.328. Quella di output la specifichiamo noi, ho messo a caso 300.
classifier.add(Dense(800, activation = 'relu'))

#Crediamo il classificatore che preso l'output della precedente iterazione assegna effettivamente una classe. 28 nel nostro caso è il numero di risposte possibili 
#(tutte le lettere + caso nullo + carattere di spazio + carattere di delete) = 28
classifier.add(Dense(29, activation = 'sigmoid'))

#COMPILING
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print("Done")

Done


In [7]:
from keras.preprocessing.image import ImageDataGenerator


#DATA PREPARATION FOR THE CNN
train_datagen = ImageDataGenerator(zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(horizontal_flip = True)


training_set = train_datagen.flow_from_directory('Dataset/asl_alphabet_train/asl_alphabet_train', 
                                                 target_size = (200, 200),
                                                 batch_size = 92,
                                                 class_mode = 'categorical')

testing_set = test_datagen.flow_from_directory('Dataset/asl_alphabet_test/asl_alphabet_test', 
                                                 target_size = (200, 200),
                                                 batch_size = 92,
                                                 class_mode = 'categorical')

print("Done")

Found 87008 images belonging to 29 classes.
Found 29 images belonging to 29 classes.
Done


In [ ]:
print("Training is about to start")

data_history = classifier.fit(training_set, 
                              steps_per_epoch = 320,
                              epochs = 32,
                              validation_data = testing_set,
                              validation_steps = 29)


print("DONE!!!")

Training is about to start
Epoch 1/32
191/320 [================>.............] - ETA: 2:26 - loss: 1.0079 - accuracy: 0.0503

In [5]:
print("Saving...")

classifier.save_weights('weights.h5')

print("Saved!")

Saving...
Saved!


In [6]:
classifier.load_weights("weights.h5")

print("Loaded")

Loaded
